In [1]:
from DataCollector import importNotDetectData
# Retrieve data from originData
import platform

data = importNotDetectData(2101, 2201)
print(platform.platform())
print('Total size of data is... %d' % len(data))

c:\users\shion\pycharmprojects\projectroute112021\venv\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


Windows-10-10.0.22000-SP0
Total size of data is... 20772


<hr>

# destIPでの分類
ハニーポットの総数は22個であることがわかった。

not detectアクセスデータに関しては

In [3]:
# ここでは、destination_ipでデータを分類している。
from DataGrouper import groupByDestIp

# ipアドレスで仕分ける
geoIps = groupByDestIp(data)
# とりあえず結果を見てみたい
print('Total Num of Variation in GeoIp: %d' % len(geoIps))
print('IP'.center(15), "total")
for ip, hit in geoIps.items():
    print(str(ip).rjust(15), str(len(hit)).rjust(5))

Total Num of Variation in GeoIp: 22
       IP       total
104.238.185.103   303
 157.245.204.40   155
    3.6.183.235  1301
  3.126.200.153  2179
  44.227.53.230  5659
155.138.134.102  1175
   139.59.35.98   187
  13.211.95.147  1121
 13.113.114.211   210
  199.247.11.85   191
 13.209.159.235   408
  45.32.119.106   192
 18.229.242.225   929
 15.185.134.185   171
   13.49.30.108   302
 52.213.219.192   348
 18.162.103.211   190
    45.32.28.50   238
   52.74.78.244   950
 165.22.204.232   268
  133.34.169.17   327
   103.81.85.89   897


# Signature ベースでの分類
Signatureごとに仕分けた上で、

受信IPにおいて同一IPからのアクセスがあるか確認したい
(受信IPはdestination_ip)
## 結果
合計22のハニーポットが存在する中で、2つ以上で同じシグネチャーのアクセスが観測できれば、「流行している攻撃」と解釈することが可能であるが、

非常に多くのnot detectedについて流行していることが確認された

→既知のものが多く含まれている可能性が高い

In [2]:
from DataGrouper import groupBySignature

#ここでは、同じsignature内に同一のipからのアクセスがどれくらいあるか確認する。
dataS = groupBySignature(data)
# result1はsignatureごとに分類した上に同一destination_ipについて整理したもの
result1 = {}
for s, hits in dataS.items():
    #それぞれのIPについて分類する。
    resultIps = {}
    for hit in hits:
        if not hit['_source']['destination_ip'] in resultIps:
            resultIps[hit['_source']['destination_ip']] = []
        resultIps[hit['_source']['destination_ip']].append(hit)
    # Signature s についての分類が完了
    # データとして格納
    result1[s] = resultIps
# result1を並び変える。
# len(x[1])refers to the size of the variation of destination_ip
# ↑ if this value is bigger than 1, it means that it is prevalent
result1 = sorted(result1.items(), key=lambda x: len(dataS[x[0]]), reverse=True)
result1 = sorted(result1, key=lambda x: len(x[1]), reverse=True)
# Print and Export result1 as "signatures_by_num_destIP(1-10).csv"
f = open("resultData/signatures_by_num_destIP.csv", "wb")
# 書き出す項目について整理しておく
print("signature".center(50), "Spots", "Counts")
f.write('Signature,Spots,Hits\n'.encode('utf-8'))
print(len(result1))
for s, r in result1:
    print(s[:50].ljust(50), str(len(r)).rjust(5), str(len(dataS[s])).rjust(6))
    b = (s + ',' + str(len(r)) + ',' + str(len(dataS[s])) + '\n').encode('utf-8')
    f.write(b)
f.flush()
f.close()

                    signature                      Spots Counts
371
/goform/setUsbUnload/.js                              22   1117
/pages/createpage-entervariables.action               22    458
/backupmgt/localJob.php                               22    281
/v1.40/containers/create                              22    256
/                                                     21    203
/include/makecvs.php                                  21     95
/_config/query_servers/cmd                            21     79
/body.txt                                             21     27
/console/css/%252e%252e%252fconsole.portal            20     60
/tools.cgi                                            20     41
/checkValid                                           19    592
/console/css/%252e%252e/consolejndi.portal            19     42
/cgibin/mainfunction.cgi&action=login&keyPath=wget    19     19
/cgibin/mainfunction.cgi&action=login&keyPath=wget    19     19
/pages/doenterpagevariables.action  

# データの整合性
データの整合性について気になったので以下で確認してみた。

not detectではポットからポット自身に送っているものはないというが、どこかで確認した気がする。
## 結論

not detectラベルがついたデータの中では、
- source_ipとgeoip>ipは完全に一致
- destination_ipとsource_ipが同じであるものは見つからなかった←(おそらくElasticSearchをいじっている中で同じものに出くわした覚えがあるのでこれについては追加で調査したい)

In [5]:
from DataOutputter import *
import importlib

result2 = []
result3 = []
for d in data:
    if 'ip' in d['_source']['geoip']:
        if not str(d['_source']['source_ip']) == str(d['_source']['geoip']['ip']):
            result2.append(d)
        if str(d['_source']['destination_ip']) == str(d['_source']['geoip']['ip']):
            result2.append(d)
    else:
        result3.append(d)
print(len(result2))
print(len(result3))
easyOutput(data=result3, folder='Unknown')

0
33
Skipped. Folder 'Unknown' already exists in output


# グラフ化
以下のコードでは、7日ごとに全データを集計した。

In [9]:
from Grapher import *
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = [12, 3]
starting = pd.to_datetime("2021/1/1 00:00:00")
ending = pd.to_datetime("2021/12/31 00:00:00")
visualizeTimeData(data, starting=starting.timestamp(), ending=ending.timestamp(), title='not detect 全データの期間分布(7日ごと)',
                  loc="not detect 全データの期間分布")

8.27から9.2までのデータが気になる...

In [20]:
starting = pd.to_datetime("2021/8/27 00:00:00")
ending = pd.to_datetime("2021/9/3 00:00:00")
visualizeTimeData(data, starting=starting.timestamp(), ending=ending.timestamp(), title='not detect 8.27~9.2の詳細(8時間ごと)',
                  loc='8.27~9.2の調査',
                  period=8 * 60 * 60, x_axis='hour')

もう少し絞って見てみる

In [6]:
from Grapher import *
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = [8, 3]
starting = pd.to_datetime("2021/8/30 8:00:00")
ending = pd.to_datetime("2021/8/31 16:00:00")
visualizeTimeData(records=data, starting=starting.timestamp(), ending=ending.timestamp(),
                  title='not detect 8.30~8.31の詳細(1時間ごと)', loc='8.27~9.2の調査',
                  period=1 * 60 * 60, x_axis='hour')

以下のコードではシグネチャーごとのデータの可視化を行っている。

In [7]:
import concurrent
from Grapher import *
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor

plt.rcParams["figure.figsize"] = [12, 3]
starting = pd.to_datetime("2021/1/1 00:00:00")
ending = pd.to_datetime("2021/12/31 00:00:00")

with ProcessPoolExecutor(5) as e:
    def task(s, d):
        visualizeTimeData(d, starting=starting.timestamp(), ending=ending.timestamp(), title=str(s) + 'の期間分布',
                          loc='Signatureごとの期間分布')
        plt.close()
    for s, d in dataS.items():
        e.submit(task(s, d))

↓パスごとの統計結果

In [ ]:
counts = []
for s, d in dataS.items():
    counts.append(len(d))
counts = sorted(counts, reverse=True)
print(counts)
signatureRoundGraph(keys=None, data=counts)


